In [1]:
import matplotlib.pyplot as plt
import dill
import sympy
from tqdm import tqdm
from scipy.sparse import lil_matrix, csc_matrix
from scipy.sparse.linalg import spsolve
import pypardiso
import numpy as np

### Load precomputed integrals

In [2]:
lagrange_linear_poisson_matrix = dill.load(
    open("../calculations/3d_lagrange_linear_poisson_matrix", "rb")
)
lagrange_linear_poisson_right_values = dill.load(
    open("../calculations/3d_lagrange_linear_poisson_right_values", "rb")
)

### Initialize the domain

In [27]:
import gmsh

# Initialize the Gmsh Python API
gmsh.initialize()

# Start a new model
gmsh.model.add("Sphere")

# Define the sphere
radius = 1.0
x_center, y_center, z_center = 0.0, 0.0, 0.0
sphere = gmsh.model.occ.addSphere(x_center, y_center, z_center, radius)

# Synchronize the data from the built-in CAD kernel with the Gmsh model
gmsh.model.occ.synchronize()

boundary = gmsh.model.getBoundary([(3, sphere)], combined=False, oriented=False, recursive=False)
sphere_surface = gmsh.model.addPhysicalGroup(2, boundary[-1])
sphere_volume = gmsh.model.addPhysicalGroup(3, [sphere])


# Define the mesh size
MeshSize = 0.1

# Set the mesh size
gmsh.model.mesh.setSize(gmsh.model.getEntities(0), MeshSize)

# Generate 3D mesh
gmsh.model.mesh.generate(3)

# Finalize the Gmsh Python API
# gmsh.fltk.run()


# tetrahedrons

In [28]:
boundary_node_tags, boundary_node_coords = gmsh.model.mesh.getNodesForPhysicalGroup(dim=2, tag=sphere_surface)
surface_node_tags, points = gmsh.model.mesh.getNodesForPhysicalGroup(dim=3, tag=sphere_volume)
points = points.reshape(-1, 3)
boundary_node_tags = boundary_node_tags - 1

In [29]:
element_types, element_tags, element_node_tags = gmsh.model.mesh.getElements()
tetrahedrons = [nodes for elem_type, nodes in zip(element_types, element_node_tags) if elem_type == 4][0]
tetrahedrons = tetrahedrons.reshape(-1, 4)
tetrahedrons = tetrahedrons - 1

In [30]:
vertex_marker_is_boundary = np.zeros(surface_node_tags.shape[0])
vertex_marker_is_boundary[boundary_node_tags] = 1

In [31]:
x, y, z = sympy.symbols("x y z")

In [32]:
F = sympy.Float(1)
right_vals = np.zeros((points.shape[0]))
right_vals[:] = sympy.lambdify((x, y, z), F)(*points.T)

G = sympy.Piecewise((1, x > 0), (-1, x>0), (0, True))*0
right_vals_G = np.zeros((points.shape[0]))
right_vals_G[:] = sympy.lambdify((x, y, z), G)(*points.T)

In [33]:
batched_tetrahedrons = points[tetrahedrons]

In [34]:
bilinear_from = lagrange_linear_poisson_matrix(
                                    batched_tetrahedrons[:, 0, 0],
                                    batched_tetrahedrons[:, 0, 1],
                                    batched_tetrahedrons[:, 0, 2],
                                    batched_tetrahedrons[:, 1, 0],
                                    batched_tetrahedrons[:, 1, 1],
                                    batched_tetrahedrons[:, 1, 2],
                                    batched_tetrahedrons[:, 2, 0],
                                    batched_tetrahedrons[:, 2, 1],
                                    batched_tetrahedrons[:, 2, 2],
                                    batched_tetrahedrons[:, 3, 0],
                                    batched_tetrahedrons[:, 3, 1],
                                    batched_tetrahedrons[:, 3, 2],
                                    ).transpose(2, 0, 1)

righ_hand_side = lagrange_linear_poisson_right_values(
                                                        batched_tetrahedrons[:, 0, 0],
                                                        batched_tetrahedrons[:, 0, 1],
                                                        batched_tetrahedrons[:, 0, 2],
                                                        batched_tetrahedrons[:, 1, 0],
                                                        batched_tetrahedrons[:, 1, 1],
                                                        batched_tetrahedrons[:, 1, 2],
                                                        batched_tetrahedrons[:, 2, 0],
                                                        batched_tetrahedrons[:, 2, 1],
                                                        batched_tetrahedrons[:, 2, 2],
                                                        batched_tetrahedrons[:, 3, 0],
                                                        batched_tetrahedrons[:, 3, 1],
                                                        batched_tetrahedrons[:, 3, 2],
                                                        right_vals[tetrahedrons[:, 0]],
                                                        right_vals[tetrahedrons[:, 1]],
                                                        right_vals[tetrahedrons[:, 2]],
                                                        right_vals[tetrahedrons[:, 3]],
                                        ).squeeze().T

In [35]:
b = np.zeros((points.shape[0]), dtype=np.float64)

idx, local_point, i = np.meshgrid(np.arange(tetrahedrons.shape[0]), np.arange(4), np.arange(4), indexing='ij')

row_indices = tetrahedrons[idx, local_point].ravel()
col_indices = tetrahedrons[idx, i].ravel()
values = bilinear_from[idx, local_point, i].ravel()

mask = (vertex_marker_is_boundary[tetrahedrons[idx, local_point]].ravel() == 0) & (vertex_marker_is_boundary[tetrahedrons[idx, i]].ravel() == 0)
m_mask = vertex_marker_is_boundary[tetrahedrons[idx, local_point]].ravel() == 1

np.subtract.at(b, col_indices[m_mask], right_vals_G[row_indices[m_mask]] * values[m_mask])

V = np.concatenate([np.ones(int(vertex_marker_is_boundary.sum())), values[mask]])
R = np.concatenate([np.where(vertex_marker_is_boundary == 1)[0], row_indices[mask]])
C = np.concatenate([np.where(vertex_marker_is_boundary == 1)[0], col_indices[mask]])

matrix = csc_matrix((V, (R, C)), shape=(points.shape[0], points.shape[0]), dtype=np.float64)


idx, local_point = np.meshgrid(np.arange(tetrahedrons.shape[0]), np.arange(4), indexing='ij')

row_indices = tetrahedrons[idx, local_point].ravel()
values = righ_hand_side[idx, local_point].ravel()
mask = vertex_marker_is_boundary[tetrahedrons[idx, local_point]].ravel() == 0
v_mask = vertex_marker_is_boundary == 1

np.subtract.at(b, row_indices[mask], values[mask])

b[v_mask] = right_vals_G[v_mask]

In [36]:
F = pypardiso.spsolve(matrix.tocsc(), b)

In [37]:
X = points[:, 0]
Y = points[:, 1]
Z = points[:, 2]

In [40]:
import plotly.graph_objects as go
import numpy as np


fig = go.Figure(
                data=[
                        go.Scatter3d(
                                    x=X, y=Y, z=Z,
                                    mode='markers',
                                    marker=dict(
                                                    size=2,
                                                    color=F,                # set color to an array/list of desired values
                                                    colorscale='Magma',   # choose a colorscale
                                                    opacity=0.1
                                                )
                                    )
                      ]
                )
fig.show()